In [2]:
import os
import sys
import numpy as np
import nibabel as nib
import pydicom
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from numpy import zeros, newaxis
from scipy.stats import norm
import cv2
import scipy.signal as signal
from pydicom.pixel_data_handlers.util import apply_color_lut
from skimage.filters import threshold_multiotsu
from skimage.filters import threshold_otsu, threshold_local
import scipy.io
import pandas as pd

In [3]:
def plot_data(data, axis=0):
    if axis==0:
        for i in range(0, len(data[0,0,:])):
            img=data[:,:,i]
            plt.imshow(img)
            plt.show()
    elif axis==1:
        for i in range(0, len(data[0,:,0])):
            img=data[:,i,:]
            plt.imshow(img)
            plt.show()
    elif axis==2:
        for i in range(0, len(data[:,0,0])):
            img=data[i,:,:]
            plt.imshow(img)
            plt.show()

In [57]:
def ch_coordinates(bottom, top, nchannels, vox_res):
#     ch_coord=np.linspace(pstart, pstop, nchannels).astype('int')
    unitvec=top-bottom
    unitvec=unitvec/np.linalg.norm(unitvec)
    
    ch_coord=np.zeros((nchannels,3))
    for i in range(nchannels):
        ch_coord[i,:]=np.round(bottom+(i*unitvec*50/vox_res))
    
    return (ch_coord).astype('int'), unitvec

In [34]:
atlas_path='/Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_atlas_v4.nii.gz'
nii_atlas=nib.load(atlas_path)
atlas=np.asanyarray(nii_atlas.dataobj)

In [51]:
labels_path='./atlas_labels.rtf'
labels=[]
anats=[]
with open(labels_path) as f:
    lines = f.readlines()
    for i,line in enumerate(lines):
        if i>6 and i%2==1:
            if i==7:
                anat= ' '.join(line.split('\\cf')[1].split()[8:])
#                 label=[line.split('\\cf')[1].split()[0], anat]
                anats.append(anat.split("\"")[1])
                labels.append(int(line.split('\\cf')[1].split()[0]))
#                 print(label)
            else:
                anat=' '.join(line.split()[7:])
#                 label=[line.split()[0], anat]
                anats.append(anat.split("\"")[1])
                labels.append(int(line.split()[0]))
#                 print(label)
                
df=pd.DataFrame()
df["Labels"]=labels
df["Anatomical Regions"]=anats

In [52]:
df

,Labels,Anatomical Regions
0,0,Clear Label
1,1,corticofugal tract and corona radiata
2,3,Subthalamic nucleus
3,4,Molecular cell layer of the cerebellum
4,5,"Cerebellum, unspecified"
...,...,...
218,444,Temporal association cortex
219,448,"Secondary visual area, medial part"
220,500,Endopiriform nucleus
221,501,"Amygdaloid area, unspecified"


In [54]:
siteMap = np.flip(np.reshape(scipy.io.loadmat('right_hemi_sitemap.mat')['siteMap'], -1))

In [15]:
#rEO-05 right hemi
top=np.array([306, 572, 434])
bottom=np.array([304, 565, 314])

ch_coord, unitvec=ch_coordinates(bottom, top, 64)

In [8]:
electrode_trajectory=np.zeros(np.shape(atlas))

In [64]:
animal_id="rEO_06"
with open(animal_id+'-chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        label=atlas[coord[0],coord[1],coord[2]]
        electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="Site:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label)
        print(line)
        f.write(line)
        f.write('\n')
    #     print('Segment: '+str(label))

Site:63 in Posterior thalamic nucleus Segment: 230.0
Site:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:57 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:4 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:5 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:55 in Laterodorsal thalamic nucleus

In [34]:
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_atlas.affine)
nib.save(nii_trajectory, 'rEO_05_electrode_trajectory_segmentation.nii.gz')

In [58]:
eo06_20240201_bottom=np.array([108, 113, 200])
eo06_20240201_top=np.array([ 118, 182, 193])

ch_coord, unitvec=ch_coordinates(eo06_20240201_bottom, eo06_20240201_top, 64, 60)

In [31]:
atlas_path='/Users/mri_registration/SAMRI/samri_output/rEO_06/segmentation_rEO-06.nii.gz'
nii_eo06_atlas=nib.load(atlas_path)
eo06_atlas=np.asanyarray(nii_eo06_atlas.dataobj)

In [60]:
eo06_path='/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-presurgery/anat/sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_00_resampled.nii.gz'
nii_eo06=nib.load(eo06_path)
eo06_img=np.asanyarray(nii_eo06.dataobj)

In [59]:
atlas=eo06_atlas
electrode_trajectory=np.zeros(np.shape(atlas))
for idx, coord in enumerate(ch_coord):
    label=atlas[coord[0],coord[1],coord[2]]
    electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
    anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
    print("Site:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label))
#     print('Segment: '+str(label))

Site:63 in Posterior thalamic nucleus Segment: 230.0
Site:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:57 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:4 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:5 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:55 in Laterodorsal thalamic nucleus

In [61]:
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_eo06.affine)
nib.save(nii_trajectory, '/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/electrode_trajectory.nii.gz')